# Script d'insertion des données dans la table "reviews"

In [ ]:
import re
import pandas as pd
import mysql.connector

from datetime import datetime

In [ ]:
from config import config

In [ ]:
reviews_path = "data/steam_game_reviews.csv"

df_reviews = pd.read_csv(reviews_path, low_memory=False)

In [ ]:
df_reviews

### Transformation et normalisation des données

In [ ]:
current_year = str(datetime.now().year)

In [ ]:
def fix_date(date_str):
    if pd.isna(date_str):
        return None

    date_str = str(date_str).strip()

    if re.search(r"\d{4}", date_str):
        return pd.to_datetime(date_str, errors='coerce')

    try:
        return pd.to_datetime(f"{date_str} {current_year}", errors='coerce')
    except:
        return None

In [ ]:
# Interprétation et conversion des dates en format date (chaînes de caractères dans le .csv)

df_reviews['formatted_date'] = df_reviews['date'].apply(fix_date)
df_reviews['formatted_date'] = df_reviews['formatted_date'].dt.date

In [ ]:
# Conversion des nombres (chaînes de caractères ici encore) en float, en supprimant les virgules et espaces

df_reviews['hours_played'] = (
    df_reviews['hours_played']
    .astype(str)
    .str.replace(',', '')
    .replace('', None)
    .astype(float)
)

In [ ]:
# Conversion des nombres (chaînes de caractères ici encore) en entiers, en supprimant les virgules et espaces

df_reviews['helpful'] = (
    df_reviews['helpful']
    .astype(str)
    .str.replace(',', '')
    .replace('', None)
    .astype(float)
    .astype('Int64')
)

df_reviews['funny'] = (
    df_reviews['funny']
    .astype(str)
    .str.replace(',', '')
    .replace('', None)
    .astype(float)
    .astype('Int64')
)

In [ ]:
# Extraction du pseudo depuis la chaîne de caractères de l'attribut "username"

df_reviews['clean_username'] = df_reviews['username'].str.split('\n').str[0]

#### Ajout des game_id

In [ ]:
# Connexion à la base

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [ ]:
# Récupération de la map game > id

cursor.execute("SELECT id, name FROM games")
game_id_map = {game: id_ for id_, game in cursor.fetchall()}

In [ ]:
# Ajout du game_id dans le dataframe des reviews

df_reviews['game_id'] = df_reviews['game_name'].map(game_id_map).astype('Int64')

#### Ajout des user_id

In [ ]:
# Récupération de la map user > id

cursor.execute("SELECT id, username FROM users")
user_id_map = {user: id_ for id_, user in cursor.fetchall()}

In [ ]:
# Extraction du pseudo depuis la chaîne de caractères de l'attribut "username"

df_reviews['clean_username'] = df_reviews['username'].str.split('\n').str[0]

In [ ]:
# Ajout de l'user_id dans le dataframe des reviews

df_reviews['user_id'] = df_reviews['clean_username'].map(user_id_map).astype('Int64')

In [ ]:
# Fermeture de la connexion

cursor.close()
cnx.close()

### Dernières préparation des données avant insertion

In [ ]:
# Conversion des valeurs Nan, NaT en None (préférable en base de données)

df_reviews = df_reviews.where(pd.notnull(df_reviews), None)

In [ ]:
# On fait correspondre le nom et l'ordre des colonnes du dataframe avec celles de la table

df_reviews = df_reviews.rename(columns={'formatted_date': 'publication_date'})

In [ ]:
df_reviews = df_reviews[['game_id', 'user_id', 'hours_played', 'publication_date', 'review', 'recommendation', 'helpful', 'funny']]

In [ ]:
# On supprime les objets dont un (ou plusieurs) des champs listés n'est pas renseigné

df_reviews = df_reviews.dropna(subset=['game_id', 'user_id', 'publication_date', 'hours_played'])

In [ ]:
# On supprime les doublons en se servant des champs précisés

df_reviews = df_reviews.drop_duplicates(
    subset=['game_id', 'user_id', 'publication_date', 'hours_played', 'review'],
    keep='first'
)

### Insertion des données

In [ ]:
# Requête SQL

insert_query = """
    INSERT IGNORE INTO reviews
    (game_id, user_id, hours_played, publication_date, review, recommandation, helpful, funny)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Connexion à la base

cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [ ]:
# Exécution de la requête

for _, row in df_reviews.iterrows():
    values = tuple(None if pd.isna(x) else x for x in row[[
        'game_id',
        'user_id',
        'hours_played',
        'publication_date',
        'review',
        'recommendation',
        'helpful',
        'funny'
    ]])
    cursor.execute(insert_query, values)

cnx.commit()

In [ ]:
# Fermeture de la connexion

cursor.close()
cnx.close()